# Necssary libary

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import unicodedata
import torch.optim as optim
from tqdm import tqdm
import wandb
import pandas as pd
import random

# path for folders

In [2]:
train_path = '/kaggle/input/dakshina/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv'
valid_path = '/kaggle/input/dakshina/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv'
test_path  = '/kaggle/input/dakshina/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv'


In [3]:
import wandb
wandb.login(key="594642013968a68e466138e783dcece6765c43b9")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bgorai005 (bgorai005-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Encoder, decoder and seq2seq model class

In [4]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, num_layers=1, cell_type='LSTM', dropout=0.2):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.embedding = nn.Embedding(input_size, embedding_dim)
        rnn_class = {'RNN': nn.RNN, 'LSTM': nn.LSTM, 'GRU': nn.GRU}[cell_type]
        self.rnn = rnn_class(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_seq):
        embedded = self.dropout(self.embedding(input_seq))
        batch_size = input_seq.size(0)
        device = input_seq.device
        if self.cell_type == 'LSTM':
            hidden = (torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device),
                     torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device))
        else:
            hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        output, hidden = self.rnn(embedded, hidden)
        return output, hidden

class Decoder(nn.Module):
    def __init__(self, output_size, embedding_dim, hidden_size, num_layers=1, cell_type='LSTM', dropout=0.2):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.embedding = nn.Embedding(output_size, embedding_dim)
        rnn_class = {'RNN': nn.RNN, 'LSTM': nn.LSTM, 'GRU': nn.GRU}[cell_type]
        self.rnn = rnn_class(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.LogSoftmax(dim=-1)

    def forward(self, input_char, hidden):
        embedded = self.dropout(self.embedding(input_char))
        output, hidden = self.rnn(embedded, hidden)
        output = self.softmax(self.out(output.squeeze(1)))
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.size(0)
        target_len = target.size(1)
        target_vocab_size = self.decoder.embedding.num_embeddings
        device = source.device
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(device)
        _, hidden = self.encoder(source)
        if self.encoder.num_layers != self.decoder.num_layers:
            if self.encoder.cell_type == 'LSTM':
                h_n, c_n = hidden
                if self.decoder.num_layers > self.encoder.num_layers:
                    extra_layers = self.decoder.num_layers - self.encoder.num_layers
                    extra_h = torch.zeros(extra_layers, batch_size, self.decoder.hidden_size).to(device)
                    extra_c = torch.zeros(extra_layers, batch_size, self.decoder.hidden_size).to(device)
                    h_n = torch.cat([h_n, extra_h], dim=0)
                    c_n = torch.cat([c_n, extra_c], dim=0)
                else:
                    h_n = h_n[:self.decoder.num_layers]
                    c_n = c_n[:self.decoder.num_layers]
                hidden = (h_n, c_n)
            else:
                if self.decoder.num_layers > self.encoder.num_layers:
                    extra_layers = self.decoder.num_layers - self.encoder.num_layers
                    extra_h = torch.zeros(extra_layers, batch_size, self.decoder.hidden_size).to(device)
                    hidden = torch.cat([hidden, extra_h], dim=0)
                else:
                    hidden = hidden[:self.decoder.num_layers]
        decoder_input = target[:, 0].unsqueeze(1)
        for t in range(1, target_len):
            output, hidden = self.decoder(decoder_input, hidden)
            outputs[:, t] = output
            teacher_force = torch.rand(1).item() < teacher_forcing_ratio
            top1 = output.argmax(1)
            decoder_input = target[:, t].unsqueeze(1) if teacher_force else top1.unsqueeze(1)
        return outputs

    def predict(self, src, max_len=30, beam_size=3):
        self.eval()
        batch_size = src.size(0)
        device = src.device
        _, hidden = self.encoder(src)
        outputs = []
        for i in range(batch_size):
            if self.encoder.cell_type == 'LSTM':
                h = hidden[0][:, i:i+1].contiguous()
                c = hidden[1][:, i:i+1].contiguous()
                hidden_state = (h, c)
            else:
                hidden_state = hidden[:, i:i+1].contiguous()
            if self.encoder.num_layers != self.decoder.num_layers:
                if self.encoder.cell_type == 'LSTM':
                    h_n, c_n = hidden_state
                    if self.decoder.num_layers > self.encoder.num_layers:
                        extra_layers = self.decoder.num_layers - self.encoder.num_layers
                        extra_h = torch.zeros(extra_layers, 1, self.decoder.hidden_size).to(device)
                        extra_c = torch.zeros(extra_layers, 1, self.decoder.hidden_size).to(device)
                        h_n = torch.cat([h_n, extra_h], dim=0)
                        c_n = torch.cat([c_n, extra_c], dim=0)
                    else:
                        h_n = h_n[:self.decoder.num_layers]
                        c_n = c_n[:self.decoder.num_layers]
                    hidden_state = (h_n, c_n)
                else:
                    if self.decoder.num_layers > self.encoder.num_layers:
                        extra_layers = self.decoder.num_layers - self.encoder.num_layers
                        extra_h = torch.zeros(extra_layers, 1, self.decoder.hidden_size).to(device)
                        hidden_state = torch.cat([hidden_state, extra_h], dim=0)
                    else:
                        hidden_state = hidden_state[:self.decoder.num_layers]
            beams = [(torch.tensor([1], device=device), 0.0, hidden_state)]  # [sequence, score, hidden]
            for _ in range(max_len):
                new_beams = []
                for seq, score, h in beams:
                    input_char = seq[-1].unsqueeze(0).unsqueeze(0)
                    output, h_new = self.decoder(input_char, h)
                    probs = torch.log_softmax(output, dim=-1).squeeze(0)
                    topk = torch.topk(probs, beam_size)
                    for idx, prob in zip(topk.indices, topk.values):
                        new_seq = torch.cat([seq, idx.unsqueeze(0)])
                        new_score = score + prob.item()
                        if self.decoder.cell_type == 'LSTM':
                            h_new = (h_new[0].contiguous(), h_new[1].contiguous())
                        else:
                            h_new = h_new.contiguous()
                        new_beams.append((new_seq, new_score, h_new))
                beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
                if beams[0][0][-1].item() == 2:  # Stop if <EOS>
                    break
            outputs.append(beams[0][0][1:])  # Exclude <SOS>
        return outputs


# data prepration class 

In [5]:
class DataPreprocessor:
    def __init__(self, batch_size=32, device='cpu'):
        self.batch_size = batch_size
        self.device = device
        self.src_vocab = None
        self.tgt_vocab = None
        self.PAD_TOKEN = '<PAD>'
        self.SOS_TOKEN = '<SOS>'
        self.EOS_TOKEN = '<EOS>'
        self.UNK_TOKEN = '<UNK>'
        self.PAD_IDX = 0
        self.SOS_IDX = 1
        self.EOS_IDX = 2
        self.UNK_IDX = 3

    def normalize_string(self, s):
        s = unicodedata.normalize('NFC', str(s))
        if all(ord(c) < 128 for c in s):
            s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
            s = s.lower()
        return s.strip()

    def load_dataset(self, file_path=None, data_frame=None):
        if file_path:
            try:
                data = pd.read_csv(file_path, sep='\t', header=None)
            except:
                data = pd.read_csv(file_path, header=None)
        elif data_frame is not None:
            data = data_frame.copy()
        else:
            raise ValueError("Either file_path or data_frame must be provided.")
        data = data.rename(columns={0: 'tgt', 1: 'src'})
        data['src'] = data['src'].apply(self.normalize_string)
        data['tgt'] = data['tgt'].apply(self.normalize_string)
        return data

    def create_vocab(self, data, column):
        vocab = {self.PAD_TOKEN: self.PAD_IDX, self.SOS_TOKEN: self.SOS_IDX,
                 self.EOS_TOKEN: self.EOS_IDX, self.UNK_TOKEN: self.UNK_IDX}
        for seq in data[column]:
            if pd.notna(seq):
                for char in seq:
                    if char not in vocab:
                        vocab[char] = len(vocab)
        return vocab

    def build_vocabularies(self, train_data):
        self.src_vocab = self.create_vocab(train_data, 'src')
        self.tgt_vocab = self.create_vocab(train_data, 'tgt')
        return self.src_vocab, self.tgt_vocab

    class TranslationDataset(Dataset):
        def __init__(self, data, input_vocab, output_vocab):
            self.data = data
            self.input_vocab = input_vocab
            self.output_vocab = output_vocab

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            src = [self.input_vocab.get(c, self.input_vocab['<UNK>']) for c in self.data.iloc[idx, 1]] + [self.input_vocab['<EOS>']]
            tgt = [self.output_vocab['<SOS>']] + [self.output_vocab.get(c, self.output_vocab['<UNK>']) for c in self.data.iloc[idx, 0]] + [self.output_vocab['<EOS>']]
            src_str = self.data.iloc[idx, 1]
            tgt_str = self.data.iloc[idx, 0]
            return torch.tensor(src, dtype=torch.long), torch.tensor(tgt, dtype=torch.long), src_str, tgt_str

    def pad_collate(self, batch):
        src_batch, tgt_batch, src_strs, tgt_strs = zip(*batch)
        src_padded = pad_sequence(src_batch, batch_first=True, padding_value=self.PAD_IDX)
        tgt_padded = pad_sequence(tgt_batch, batch_first=True, padding_value=self.PAD_IDX)
        return src_padded, tgt_padded, list(src_strs), list(tgt_strs)

    def prepare_data(self, train_data, val_data, test_data):
        if self.src_vocab is None or self.tgt_vocab is None:
            self.build_vocabularies(train_data)
        train_dataset = self.TranslationDataset(train_data, self.src_vocab, self.tgt_vocab)
        val_dataset = self.TranslationDataset(val_data, self.src_vocab, self.tgt_vocab)
        test_dataset = self.TranslationDataset(test_data, self.src_vocab, self.tgt_vocab)
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True,
                                 collate_fn=self.pad_collate, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False,
                                collate_fn=self.pad_collate, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False,
                                 collate_fn=self.pad_collate, pin_memory=True)
        return train_loader, val_loader, test_loader
import torch
import torch.nn as nn



# train class 

In [6]:
class Trainer:
    def __init__(self, model, train_loader, val_loader, config, device='cpu', save_path='best_model.pt'):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        
        self.device = device
        self.config = config
        self.teacher_forcing_ratio = config.teacher_forcing
        self.num_epochs = config.epochs
        self.save_path = save_path
        self.criterion = nn.CrossEntropyLoss(ignore_index=0)  # Changed to CrossEntropyLoss
        self.optimizer = optim.Adam(self.model.parameters(), lr=config.learning_rate)
        self.src_vocab = None  # To store vocab for predictions
        self.tgt_vocab = None

    def compute_token_accuracy(self, outputs, targets):
        """Compute token-level accuracy."""
        outputs = outputs.argmax(dim=-1)  # [batch_size, seq_len]
        non_pad_mask = (targets != 0) & (targets != 1) & (targets != 2)  # Exclude <PAD>, <SOS>, <EOS>
        correct = (outputs == targets) & non_pad_mask
        total = non_pad_mask.sum().item()
        correct = correct.sum().item()
        return correct / total if total > 0 else 0.0

    def compute_sequence_accuracy(self, outputs, targets):
        """Compute sequence-level accuracy."""
        outputs = outputs.argmax(dim=-1)  # [batch_size, seq_len]
        correct = 0
        total = outputs.size(0)
        for pred, tgt in zip(outputs, targets):
            # Compare sequences, ignoring <PAD>, <SOS>, <EOS>
            pred = pred[(tgt != 0) & (tgt != 1) & (tgt != 2)]
            tgt = tgt[(tgt != 0) & (tgt != 1) & (tgt != 2)]
            if torch.equal(pred, tgt):
                correct += 1
        return correct / total if total > 0 else 0.0

    def train_epoch(self):
        self.model.train()
        total_loss, total_token_acc, total_seq_acc, total_samples = 0.0, 0.0, 0.0, 0

        pbar = tqdm(self.train_loader, desc="Training", leave=False)
        for src, tgt, _, _ in pbar:  # Adjusted for src_strs, tgt_strs from DataLoader
            src, tgt = src.to(self.device), tgt.to(self.device)
            self.optimizer.zero_grad()

            output = self.model(src, tgt, self.teacher_forcing_ratio)
            output = output[:, 1:].contiguous().view(-1, output.size(-1))
            tgt_flat = tgt[:, 1:].contiguous().view(-1)

            loss = self.criterion(output, tgt_flat)
            loss.backward()
            self.optimizer.step()

            batch_size = src.size(0)
            token_acc = self.compute_token_accuracy(
                output.view(batch_size, -1, output.size(-1)), tgt[:, 1:]
            )
            seq_acc = self.compute_sequence_accuracy(
                output.view(batch_size, -1, output.size(-1)), tgt[:, 1:]
            )

            total_loss += loss.item() * batch_size
            total_token_acc += token_acc * batch_size
            total_seq_acc += seq_acc * batch_size
            total_samples += batch_size

            pbar.set_postfix(loss=loss.item(), token_acc=token_acc, seq_acc=seq_acc)

        avg_loss = total_loss / total_samples
        avg_token_acc = total_token_acc / total_samples
        avg_seq_acc = total_seq_acc / total_samples
        return avg_loss, avg_token_acc, avg_seq_acc

    def evaluate(self, loader):
        self.model.eval()
        total_loss, total_token_acc, total_seq_acc, total_samples = 0.0, 0.0, 0.0, 0

        pbar = tqdm(loader, desc="Evaluating", leave=False)
        with torch.no_grad():
            for src, tgt, _, _ in pbar:
                src, tgt = src.to(self.device), tgt.to(self.device)

                output = self.model(src, tgt, teacher_forcing_ratio=0.0)
                output = output[:, 1:].contiguous().view(-1, output.size(-1))
                tgt_flat = tgt[:, 1:].contiguous().view(-1)

                loss = self.criterion(output, tgt_flat)

                batch_size = src.size(0)
                token_acc = self.compute_token_accuracy(
                    output.view(batch_size, -1, output.size(-1)), tgt[:, 1:]
                )
                seq_acc = self.compute_sequence_accuracy(
                    output.view(batch_size, -1, output.size(-1)), tgt[:, 1:]
                )

                total_loss += loss.item() * batch_size
                total_token_acc += token_acc * batch_size
                total_seq_acc += seq_acc * batch_size
                total_samples += batch_size

                pbar.set_postfix(loss=loss.item(), token_acc=token_acc, seq_acc=seq_acc)

        avg_loss = total_loss / total_samples
        avg_token_acc = total_token_acc / total_samples
        avg_seq_acc = total_seq_acc / total_samples
        return avg_loss, avg_token_acc, avg_seq_acc

    def train(self, src_vocab, tgt_vocab):
        """Train the model, logging metrics and predictions to Wandb."""
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        best_val_seq_acc = 0.0
        patience = getattr(self.config, 'patience', 3)
        patience_counter = 0

        for epoch in range(1, self.num_epochs + 1):
            # Train
            train_loss, train_token_acc, train_seq_acc = self.train_epoch()
            # Evaluate
            val_loss, val_token_acc, val_seq_acc = self.evaluate(self.val_loader)

            # Print metrics
            print(f'\nEpoch {epoch}/{self.num_epochs}')
            print(f'Train Loss: {train_loss:.4f} | Train Token Acc: {train_token_acc*100:.2f}% | Train Seq Acc: {train_seq_acc*100:.2f}%')
            print(f'Val Loss:   {val_loss:.4f} | Val Token Acc:   {val_token_acc*100:.2f}% | Val Seq Acc:   {val_seq_acc*100:.2f}%')
            print('-' * 60)

            # Log metrics to Wandb
            wandb.log({
                'epoch': epoch,
                'train_loss': train_loss,
                 'val_loss': val_loss,
                'train_token_accuracy': train_token_acc,
                'val_token_accuracy': val_token_acc,
                'train_sequence_accuracy': train_seq_acc,
                'val_sequence_accuracy': val_seq_acc
            })

            # Log sample predictions
            src_sample, tgt_sample, src_strs, tgt_strs = next(iter(self.val_loader))
            src_sample = src_sample.to(self.device)
            preds = self.model.predict(src_sample[:5], max_len=30, beam_size=self.config.beam_size)

            inv_src_vocab = {v: k for k, v in src_vocab.items()}
            inv_tgt_vocab = {v: k for k, v in tgt_vocab.items()}
            table = wandb.Table(columns=["Input", "Target", "Prediction"])
            for i in range(len(preds)):
                input_str = ''.join([inv_src_vocab.get(id.item(), '?') for id in src_sample[i] if id.item() not in [0, src_vocab['<EOS>']]])
                target_str = ''.join([inv_tgt_vocab.get(id.item(), '?') for id in tgt_sample[i] if id.item() not in [0, tgt_vocab['<EOS>'], tgt_vocab['<SOS>']]])
                pred_str = ''.join([inv_tgt_vocab.get(id.item(), '?') for id in preds[i] if id.item() not in [0, tgt_vocab['<EOS>']]])
                table.add_data(input_str, target_str, pred_str)
            wandb.log({"predictions": table})

            # Early stopping and checkpoint
            if val_seq_acc > best_val_seq_acc:
                best_val_seq_acc = val_seq_acc
                patience_counter = 0
                torch.save(self.model.state_dict(), self.save_path)
                print(f"✅ New best model saved with val sequence accuracy: {val_seq_acc*100:.2f}%")
            else:
                patience_counter += 1
                print(f"⚠️ No improvement. Patience counter: {patience_counter}/{patience}")
                if patience_counter >= patience:
                    print(" Early stopping triggered.")
                    break


# hyper parameter tuning for searching best hyperparamter

In [ ]:
# Wandb sweep config
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'val_sequence_accuracy', 'goal': 'maximize'},
    'parameters': {
        'emb_dim': {'values': [64, 128, 256]},
        'hidden_dim': {'values': [128, 256]},
        'enc_layers': {'values': [1, 2, 3]},
        'dec_layers': {'values': [1, 2, 3]},
        'cell_type': {'values': ['LSTM', 'GRU','RNN']},
        'dropout': {'values': [0.2, 0.3, 0.4]},
        'batch_size': {'values': [32, 64, 128]},
        'learning_rate': {'values': [0.001, 0.0005, 0.0001]},
        'teacher_forcing': {'values': [0.5, 0.7, 0.9]},
        'beam_size': {'values': [1, 3, 5]},
        'patience': {'value': 3},
        'epochs': {'values': [10, 15]}
    }
}


# train loader function to handle the sweep

In [12]:
import torch
import wandb
import pandas as pd
import os

def train_loader(
    train_path='/kaggle/input/dakshina/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv',
    valid_path='/kaggle/input/dakshina/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv',
    test_path='/kaggle/input/dakshina/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv',
    device='cuda' if torch.cuda.is_available() else 'cpu',
    save_path='/kaggle/working/best_model.pt'
):
    """
    Training function for running a WandB sweep on the Bengali Dakshina dataset.
    """
    # Initialize WandB
    wandb.init(project="assignment_3")
    # Shortcut to config
    config = wandb.config

    # Construct a descriptive run name
    run_name = (
        f"-cell-{config.cell_type}"
        f"embed-{config.emb_dim}"
        f"-enc_layers-{config.enc_layers}"
        f"-dec_layers-{config.dec_layers}"
        f"-hid-{config.hidden_dim}"
       
        f"-dropout-{config.dropout}"
        f"-bs-{config.batch_size}"
        f"-lr-{config.learning_rate}"
        f"-tf-{config.teacher_forcing}"
        f"-beam-{config.beam_size}"
    )
    wandb.run.name = run_name

    
    # Initialize DataPreprocessor
    preprocessor = DataPreprocessor(batch_size=config.batch_size, device=device)

    # Load datasets
    train_data = preprocessor.load_dataset(train_path)
    val_data = preprocessor.load_dataset(valid_path)
    test_data = preprocessor.load_dataset(test_path)

    # Prepare data loaders
    train_loader, val_loader, test_loader = preprocessor.prepare_data(train_data, val_data, test_data)

    # Initialize model
    encoder = Encoder(
        input_size=len(preprocessor.src_vocab),
        embedding_dim=config.emb_dim,
        hidden_size=config.hidden_dim,
        num_layers=config.enc_layers,
        cell_type=config.cell_type,
        dropout=config.dropout
    )
    decoder = Decoder(
        output_size=len(preprocessor.tgt_vocab),
        embedding_dim=config.emb_dim,
        hidden_size=config.hidden_dim,
        num_layers=config.dec_layers,
        cell_type=config.cell_type,
        dropout=config.dropout
    )
    model = Seq2Seq(encoder, decoder).to(device)

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        config=config,
        device=device,
        save_path=save_path
    )

    # Train with vocabularies
    trainer.train(preprocessor.src_vocab, preprocessor.tgt_vocab)

   
    # Finish Wandb run
    wandb.finish()


if __name__ == "__main__":
    sweep_id = wandb.sweep(sweep_config, project="assignment_3")
    wandb.agent(sweep_id, function=train_loader, count=10)


Create sweep with ID: bskdv1xl
Sweep URL: https://wandb.ai/bgorai005-iit-madras/assignment_3/sweeps/bskdv1xl


wandb: Agent Starting Run: w2xdfoci with config:
wandb: 	batch_size: 64
wandb: 	beam_size: 5
wandb: 	cell_type: LSTM
wandb: 	dec_layers: 3
wandb: 	dropout: 0.3
wandb: 	emb_dim: 256
wandb: 	enc_layers: 1
wandb: 	epochs: 15
wandb: 	hidden_dim: 128
wandb: 	learning_rate: 0.0005
wandb: 	patience: 3
wandb: 	teacher_forcing: 0.5


Training:  12%|█▏        | 179/1478 [00:11<01:17, 16.82it/s, loss=3.26, seq_acc=0, token_acc=0.118] wandb: Ctrl + C detected. Stopping sweep.


In [8]:
import torch
import pandas as pd
from tqdm import tqdm
import os

class TestEvaluator:
    def __init__(self, model, test_loader, src_vocab, tgt_vocab, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model.to(device)
        self.test_loader = test_loader
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.device = device
        self.inv_src_vocab = {v: k for k, v in src_vocab.items()}
        self.inv_tgt_vocab = {v: k for k, v in tgt_vocab.items()}
    def check_model_file(self, model_path):
        """Check if the model file exists and provide guidance if it doesn't."""
        if not os.path.exists(model_path):
            error_msg = f"Error: Model file not found at '{model_path}'.\n"
            error_msg += "Possible solutions:\n"
            error_msg += "1. Ensure training completed successfully and saved the model to '/kaggle/working/best_model.pt'.\n"
            error_msg += "2. Check if the model was saved to a different path and update 'model_path'.\n"
            error_msg += "3. Rerun the training script to generate the model.\n"
            error_msg += "4. If running in Kaggle, verify that '/kaggle/working/' is accessible and the file was persisted.\n"
            error_msg += "5. Provide the correct path to an existing model file."
            raise FileNotFoundError(error_msg)
        print(f"Model file found at '{model_path}'.")
    def compute_sequence_accuracy(self, outputs, targets):
        """Compute sequence-level accuracy (exact match, ignoring special tokens)."""
        outputs = outputs.argmax(dim=-1)  # [batch_size, seq_len]
        correct = 0
        total = outputs.size(0)
        for pred, tgt in zip(outputs, targets):
            pred = pred[(tgt != 0) & (tgt != 1) & (tgt != 2)]  # Exclude <PAD>, <SOS>, <EOS>
            tgt = tgt[(tgt != 0) & (tgt != 1) & (tgt != 2)]
            if torch.equal(pred, tgt):
                correct += 1
        return correct / total if total > 0 else 0.0

    def evaluate_test_set(self):
        """Evaluate the model on the test set and return sequence accuracy."""
        self.model.eval()
        total_seq_acc, total_samples = 0.0, 0

        with torch.no_grad():
            for src, tgt, _, _ in tqdm(self.test_loader, desc="Evaluating Test Set"):
                src, tgt = src.to(self.device), tgt.to(self.device)
                output = self.model(src, tgt, teacher_forcing_ratio=0.0)
                output = output[:, 1:].contiguous()  # Exclude <SOS>
                seq_acc = self.compute_sequence_accuracy(output, tgt[:, 1:])
                batch_size = src.size(0)
                total_seq_acc += seq_acc * batch_size
                total_samples += batch_size

        avg_seq_acc = total_seq_acc / total_samples
        return avg_seq_acc

    def generate_predictions(self, output_dir="predictions_vanilla", num_samples=10):
        """Generate predictions for the test set and save them to a file. Return samples for display."""
        self.model.eval()
        predictions = []
        sample_data = []

        os.makedirs(output_dir, exist_ok=True)
        pred_file = os.path.join(output_dir, "predictions.tsv")

        with torch.no_grad():
            for src, tgt, src_strs, tgt_strs in tqdm(self.test_loader, desc="Generating Predictions"):
                src = src.to(self.device)
                preds = self.model.predict(src, max_len=30, beam_size=3)  # Use beam_size=3 as default
                for i in range(len(preds)):
                    input_str = src_strs[i]
                    target_str = tgt_strs[i]
                    pred_ids = preds[i]
                    pred_str = ''.join([self.inv_tgt_vocab.get(id.item(), '?') for id in pred_ids if id.item() not in [0, self.tgt_vocab['<EOS>']]])
                    predictions.append((input_str, target_str, pred_str))
                    if len(sample_data) < num_samples:
                        sample_data.append((input_str, target_str, pred_str))

        # Save all predictions to a TSV file
        pred_df = pd.DataFrame(predictions, columns=["Input", "Target", "Prediction"])
        pred_df.to_csv(pred_file, sep='\t', index=False)

        return sample_data, pred_file

    def display_samples(self, sample_data):
        """Format sample predictions as a markdown table."""
        markdown = "| Input | Target | Prediction | Match |\n"
        markdown += "|-------|--------|------------|-------|\n"
        for input_str, target_str, pred_str in sample_data:
            match = "✅" if pred_str == target_str else "❌"
            markdown += f"| {input_str} | {target_str} | {pred_str} | {match} |\n"
        return markdown

    def display_samples_highlight_incorrect(self, sample_data):
        """Format sample predictions as a markdown table, highlighting incorrect predictions."""
        markdown = "| Input | Target | Prediction | Match |\n"
        markdown += "|-------|--------|------------|-------|\n"
        for input_str, target_str, pred_str in sample_data:
            match = "✅" if pred_str == target_str else "❌"
            display_pred = f"**{pred_str}**" if pred_str != target_str else pred_str
            markdown += f"| {input_str} | {target_str} | {display_pred} | {match} |\n"
        return markdown

# class test evaluator

In [7]:

class TestEvaluator:
    def __init__(self, model, test_loader, src_vocab, tgt_vocab, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.model = model.to(device)
        self.test_loader = test_loader
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.device = device
        self.inv_src_vocab = {v: k for k, v in src_vocab.items()}
        self.inv_tgt_vocab = {v: k for k, v in tgt_vocab.items()}

    def check_model_file(self, model_path):
        """Check if the model file exists and provide guidance if it doesn't."""
        if not os.path.exists(model_path):
            error_msg = f"Error: Model file not found at '{model_path}'.\n"
            error_msg += "Possible solutions:\n"
            error_msg += "1. Ensure training completed successfully and saved the model to '/kaggle/working/best_model.pt'.\n"
            error_msg += "2. Check if the model was saved to a different path and update 'model_path'.\n"
            error_msg += "3. Rerun the training script to generate the model.\n"
            error_msg += "4. If running in Kaggle, verify that '/kaggle/working/' is accessible and the file was persisted.\n"
            error_msg += "5. Provide the correct path to an existing model file."
            raise FileNotFoundError(error_msg)
        print(f"Model file found at '{model_path}'.")

    def compute_sequence_accuracy(self, outputs, targets):
        """Compute sequence-level accuracy (exact match, ignoring special tokens)."""
        outputs = outputs.argmax(dim=-1)  # [batch_size, seq_len]
        correct = 0
        total = outputs.size(0)
        for pred, tgt in zip(outputs, targets):
            pred = pred[(tgt != 0) & (tgt != 1) & (tgt != 2)]  # Exclude <PAD>, <SOS>, <EOS>
            tgt = tgt[(tgt != 0) & (tgt != 1) & (tgt != 2)]
            if torch.equal(pred, tgt):
                correct += 1
        return correct / total if total > 0 else 0.0

    def evaluate_test_set(self):
        """Evaluate the model on the test set and return sequence accuracy."""
        self.model.eval()
        total_seq_acc, total_samples = 0.0, 0

        with torch.no_grad():
            for src, tgt, _, _ in tqdm(self.test_loader, desc="Evaluating Test Set"):
                src, tgt = src.to(self.device), tgt.to(self.device)
                output = self.model(src, tgt, teacher_forcing_ratio=0.0)
                output = output[:, 1:].contiguous()  # Exclude <SOS>
                seq_acc = self.compute_sequence_accuracy(output, tgt[:, 1:])
                batch_size = src.size(0)
                total_seq_acc += seq_acc * batch_size
                total_samples += batch_size

        avg_seq_acc = total_seq_acc / total_samples
        return avg_seq_acc

    def generate_predictions(self, output_dir="predictions     predictions_vanilla", num_samples=35):
        """Generate predictions for the entire test set, save to a TSV file, and return a random sample of predictions."""
        self.model.eval()
        predictions = []

        os.makedirs(output_dir, exist_ok=True)
        pred_file = os.path.join(output_dir, "predictions.tsv")

        with torch.no_grad():
            for src, tgt, src_strs, tgt_strs in tqdm(self.test_loader, desc="Generating Predictions"):
                src = src.to(self.device)
                preds = self.model.predict(src, max_len=30, beam_size=3)  # Use beam_size=3 as default
                for i in range(len(preds)):
                    input_str = src_strs[i]
                    target_str = tgt_strs[i]
                    pred_ids = preds[i]
                    pred_str = ''.join([self.inv_tgt_vocab.get(id.item(), '?') for id in pred_ids if id.item() not in [0, self.tgt_vocab['<EOS>']]])
                    predictions.append((input_str, target_str, pred_str))

        # Save all predictions to a TSV file
        pred_df = pd.DataFrame(predictions, columns=["Input", "Target", "Prediction"])
        pred_df.to_csv(pred_file, sep='\t', index=False)

        # Select random samples
        sample_data = random.sample(predictions, min(num_samples, len(predictions)))

        return sample_data, pred_file

    def format_sample_table(self, sample_data):
        """Format sample predictions as a markdown table with colored backgrounds."""
        markdown = "| Input | Target | Prediction | Status |\n"
        markdown += "|-------|--------|------------|--------|\n"
        for input_str, target_str, pred_str in sample_data:
            is_correct = pred_str == target_str
            bg_color = "background-color: #90EE90" if is_correct else "background-color: #FFB6C1"  # Green or Pink
            markdown += f"| {input_str} | {target_str} | <span style=\"{bg_color}\">{pred_str}</span> | {'Correct' if is_correct else 'Incorrect'} |\n"
        return markdown


# best hyper paramter

In [1]:
#best hyper parameter 

# batch_size:
#     value: 32
# beam_size:
#     value: 3
# cell_type:
#     value: LSTM
# dec_layers:
#     value: 3
# dropout:
#     value: 0.3
# emb_dim:
#     value: 64
# enc_layers:
#     value: 3
# epochs:
#     value: 15
# hidden_dim:
#     value: 256
# learning_rate:
#     value: 0.0005
# patience:
#     value: 3
# teacher_forcing:
#     value: 0.7

# train with best hyperparameter function 

In [8]:
import torch
import pandas as pd
from tqdm import tqdm
import os
import wandb
import random


def train_with_best_hyperparams(
    train_path='/kaggle/input/dakshina/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv',
    valid_path='/kaggle/input/dakshina/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv',
    test_path='/kaggle/input/dakshina/dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv',
    device='cuda' if torch.cuda.is_available() else 'cpu',
    save_path='/kaggle/working/best_model.pt'
):
    """
    Train the model with the best hyperparameters and save it to save_path.
    Returns the trained model and vocabularies.
    """
    # Best hyperparameters from WandB
    config = {
        'batch_size': 64,
        'beam_size': 1,
        'cell_type': 'GRU',
        'dec_layers': 3,
        'dropout': 0.4,
        'emb_dim': 128,
        'enc_layers': 2,
        'epochs': 10,
        'hidden_dim': 256,
        'learning_rate': 0.0005,
        'patience': 3,
        'teacher_forcing': 0.7
    }



    # Convert config to an object for compatibility with Trainer
    class Config:
        def __init__(self, params):
            for key, value in params.items():
                setattr(self, key, value)
    
    config_obj = Config(config)

    # Initialize WandB run
    wandb.init(project="assignment_3", config=config, name="best_hyperparams_run_vanilla")
    
    # Initialize DataPreprocessor
    preprocessor = DataPreprocessor(batch_size=config['batch_size'], device=device)
    
    # Load datasets
    train_data = preprocessor.load_dataset(train_path)
    val_data = preprocessor.load_dataset(valid_path)
    test_data = preprocessor.load_dataset(test_path)
    
    # Prepare data loaders
    train_loader, val_loader, test_loader = preprocessor.prepare_data(train_data, val_data, test_data)
    
    # Initialize model
    encoder = Encoder(
        input_size=len(preprocessor.src_vocab),
        embedding_dim=config['emb_dim'],
        hidden_size=config['hidden_dim'],
        num_layers=config['enc_layers'],
        cell_type=config['cell_type'],
        dropout=config['dropout']
    )
    decoder = Decoder(
        output_size=len(preprocessor.tgt_vocab),
        embedding_dim=config['emb_dim'],
        hidden_size=config['hidden_dim'],
        num_layers=config['dec_layers'],
        cell_type=config['cell_type'],
        dropout=config['dropout']
    )
    model = Seq2Seq(encoder, decoder).to(device)
    
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        config=config_obj,
        device=device,
        save_path=save_path
    )
    
    # Train the model
    trainer.train(preprocessor.src_vocab, preprocessor.tgt_vocab)
    
    # Finish WandB run
    wandb.finish()
    
    print(f"Training completed. Model saved to {save_path}")
    return model, preprocessor.src_vocab, preprocessor.tgt_vocab, test_loader


# function evaluting for best model

In [9]:
def evaluate_with_best_model(
    model,
    test_loader,
    src_vocab,
    tgt_vocab,
    model_path='/kaggle/working/best_model.pt',
    device='cuda' if torch.cuda.is_available() else 'cpu'
):
    """
    Evaluate the model on the test set and generate predictions with highlighted incorrect outputs.
    Returns sequence accuracy, markdown table, and predictions file path.
    """
    # Initialize TestEvaluator
    evaluator = TestEvaluator(model, test_loader, src_vocab, tgt_vocab, device)
    
    # Check model file
    evaluator.check_model_file(model_path)
    
    # Load the model (already loaded in model, but verify for consistency)
    model.load_state_dict(torch.load(model_path, map_location=device))
    print(f"Loaded model from {model_path}")
    
    # Evaluate on test set
    test_seq_acc = evaluator.evaluate_test_set()
    print(f"\nTest Set Sequence Accuracy: {test_seq_acc*100:.2f}%")
    
    # Generate predictions
    sample_data, pred_file = evaluator.generate_predictions(num_samples=30)
    print(f"\nPredictions saved to {pred_file}")
    
    # Display sample predictions with incorrect ones highlighted
    markdown_table = evaluator.format_sample_table(sample_data)
    print("\nSample Predictions (Incorrect Predictions Highlighted):")
    print(markdown_table)
    
    return test_seq_acc, markdown_table, pred_file

# main function for test set 

In [10]:
if __name__ == "__main__":
    try:
        # Train with best hyperparameters
        model, src_vocab, tgt_vocab, test_loader = train_with_best_hyperparams()
        
        # Evaluate on test set
        test_seq_acc, markdown_table, pred_file = evaluate_with_best_model(
            model, test_loader, src_vocab, tgt_vocab
        )
    except FileNotFoundError as e:
        print(e)
    except Exception as e:
        print(f"Error during training or evaluation: {e}")


Epoch 1/10
Train Loss: 1.9551 | Train Token Acc: 40.19% | Train Seq Acc: 3.50%
Val Loss:   1.6544 | Val Token Acc:   51.66% | Val Seq Acc:   13.31%
------------------------------------------------------------
✅ New best model saved with val sequence accuracy: 13.31%



Epoch 2/10
Train Loss: 0.9289 | Train Token Acc: 69.65% | Train Seq Acc: 16.50%
Val Loss:   1.4351 | Val Token Acc:   61.11% | Val Seq Acc:   24.66%
------------------------------------------------------------
✅ New best model saved with val sequence accuracy: 24.66%



Epoch 3/10
Train Loss: 0.7049 | Train Token Acc: 77.00% | Train Seq Acc: 25.73%
Val Loss:   1.4084 | Val Token Acc:   64.10% | Val Seq Acc:   28.90%
------------------------------------------------------------
✅ New best model saved with val sequence accuracy: 28.90%



Epoch 4/10
Train Loss: 0.5961 | Train Token Acc: 80.64% | Train Seq Acc: 32.28%
Val Loss:   1.3473 | Val Token Acc:   66.95% | Val Seq Acc:   31.87%
------------------------------------------------------------
✅ New best model saved with val sequence accuracy: 31.87%



Epoch 5/10
Train Loss: 0.5243 | Train Token Acc: 82.96% | Train Seq Acc: 37.13%
Val Loss:   1.3000 | Val Token Acc:   68.93% | Val Seq Acc:   34.56%
------------------------------------------------------------
✅ New best model saved with val sequence accuracy: 34.56%


...

Training completed. Model saved to /kaggle/working/best_model.pt
Model file found at '/kaggle/working/best_model.pt'.
Loaded model from /kaggle/working/best_model.pt


Evaluating Test Set: 100%|██████████| 145/145 [00:04<00:00, 35.99it/s]



Test Set Sequence Accuracy: 35.59%


Generating Predictions: 100%|██████████| 145/145 [01:56<00:00,  1.25it/s]


Predictions saved to predictions     predictions_vanilla/predictions.tsv

Sample Predictions (Incorrect Predictions Highlighted):
| Input | Target | Prediction | Status |
|-------|--------|------------|--------|
| ranchi | রাঁচি | <span style="background-color: #FFB6C1">রাঞ্চি</span> | Incorrect |
| potel | পটেল | <span style="background-color: #FFB6C1">পোটেল</span> | Incorrect |
| bandhuraa | বন্ধুরা | <span style="background-color: #90EE90">বন্ধুরা</span> | Correct |
| met | মত | <span style="background-color: #FFB6C1">মেট</span> | Incorrect |
| baasati | বাসাটি | <span style="background-color: #90EE90">বাসাটি</span> | Correct |
| sikkhoker | শিক্ষকের | <span style="background-color: #90EE90">শিক্ষকের</span> | Correct |
| nirangkush | নিরঙ্কুশ | <span style="background-color: #FFB6C1">নিরঙ্কুষ</span> | Incorrect |
| pyada | প্যাড | <span style="background-color: #90EE90">প্যাড</span> | Correct |
| belphast | বেলফাস্ট | <span style="background-color: #90EE90">বেলফাস্ট</span> | Correc

In [19]:
import pandas as pd

# Step 1: Load the TSV file into a DataFrame
df = pd.read_csv('/kaggle/working/predictions     predictions_vanilla/predictions.tsv', sep='\t')

# Step 2: Display the first 35 rows
print(df.head(35))

# Step 3: Convert and save the DataFrame as a CSV file
df.to_csv('/kaggle/working/predictions     predictions_vanilla/predictions.tsv', index=False)


               Input        Target    Prediction
0               anri          অঁরি      অ্যান্রি
1              aunry          অঁরি        আউন্রি
2               onry          অঁরি         অন্রি
3                ori          অঁরি           ওরি
4    angsagrohonkari  অংশগ্রহনকারী  অংশগ্রহণকারী
5   angshogrohonkari  অংশগ্রহনকারী  অংশগ্রহণকারী
6    angsogrohonkari  অংশগ্রহনকারী  অংশগ্রহণকারী
7    anshograhankari  অংশগ্রহনকারী  অংশগ্রহণকারী
8    ongsagrohonkari  অংশগ্রহনকারী  অংশগ্রহণকারী
9   ongshogrohonkari  অংশগ্রহনকারী  অংশগ্রহণকারী
10   ongsogrohonkari  অংশগ্রহনকারী  অংশগ্রহণকারী
11        akkharguli     অক্ষরগুলি     আক্ষরগুলি
12        akshorguli     অক্ষরগুলি     আক্ষরগুলি
13         aksorguli     অক্ষরগুলি     আক্ষরগুলি
14        okkhorguli     অক্ষরগুলি     অক্ষরগুলি
15        okshorguli     অক্ষরগুলি     অক্ষরগুলি
16         oksorguli     অক্ষরগুলি     অক্ষরগুলি
17        akkhargulo     অক্ষরগুলো     আক্ষরগুলো
18        akshorgulo     অক্ষরগুলো     আক্ষরগুলো
19         aksorgulo